In [ ]:
import cupy as cp
from cuml.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from itertools import combinations
from concurrent.futures import ThreadPoolExecutor
import time

# ... [previous data loading code] ...

def manual_loocv_gpu_fast(predictors, data):
    """Optimized GPU LOOCV"""
    n = len(data)
    X = cp.array(data[list(predictors)].values, dtype=cp.float32)
    y = cp.array(data['price_class'].values, dtype=cp.float32)
    
    predictions = cp.zeros(n)
    
    for i in range(n):
        # Create masks for train/test
        mask = cp.ones(n, dtype=bool)
        mask[i] = False
        
        X_train = X[mask]
        y_train = y[mask]
        X_test = X[i:i+1]
        
        # GPU-accelerated logistic regression
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        predictions[i] = model.predict_proba(X_test)[0, 1]
    
    # Calculate CV error on GPU
    cv_error = float(cp.mean((y - predictions) ** 2))
    
    return cv_error

# Parallel evaluation across models
def evaluate_model(args):
    i, predictors = args
    cv_error = manual_loocv_gpu_fast(predictors, train)
    return {
        'model_id': i + 1,
        'predictors': ', '.join(predictors),
        'num_predictors': len(predictors),
        'cv_error': cv_error
    }

print("Starting GPU-accelerated LOOCV...\n")
start_time = time.time()

# Use ThreadPoolExecutor for parallel model evaluation
with ThreadPoolExecutor(max_workers=4) as executor:
    model_results = list(executor.map(evaluate_model, enumerate(all_models)))

end_time = time.time()
print(f"\nTotal time: {end_time - start_time:.2f} seconds")

# ... [rest of results code] ...
